# SQL Проект. 

**Описание исследования:**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В
свободное время жители городов больше не выходят на улицу, не посещают кафе
и торговые центры. Зато стало больше времени для книг. Это заметили
стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг
по подписке. Ваша первая задача как аналитика — проанализировать базу
данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские
обзоры книг. Эти данные помогут сформулировать ценностное предложение для
нового продукта.


**Описание данных:**

`Таблица books` содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

`Таблица authors` содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

`Таблица publishers` cодержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

`Таблица ratings` cодержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

`Таблица reviews` cодержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**Задачи исследования:**
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставилибольше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'}

# название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Изучение общей информации.

In [3]:
query = '''

        SELECT *
        FROM books
        
        '''

books = pd.io.sql.read_sql(query, con = engine) 
books.name = 'books'

In [4]:
query = '''

        SELECT *
        FROM authors
        
        '''

authors = pd.io.sql.read_sql(query, con = engine) 
authors.name = 'authors'

In [5]:
query = '''

        SELECT *
        FROM publishers

        '''

publishers = pd.io.sql.read_sql(query, con = engine) 
publishers.name = 'publisher'

In [6]:
query = '''

        SELECT *
        FROM ratings

        '''

ratings = pd.io.sql.read_sql(query, con = engine) 
ratings.name = 'publisher'

In [7]:
query = '''

        SELECT *
        FROM reviews

        '''

reviews = pd.io.sql.read_sql(query, con = engine) 
reviews.name = 'publisher'

In [8]:
def get_info(dataframes):
    for df in dataframes:
        print(df.name)
        print('Вызов метода head()') 
        display(df.head())
        print()
        print('Вызов метода info()') 
        df.info()
        print()
        print('Вызов метода describe()')
        display(df.describe().T)
        print()
        print('Количество пропусков: {}'.format(df.isna().sum()))

In [9]:
dataframes = [books, authors, publishers, ratings, reviews]

In [10]:
get_info(dataframes)

books
Вызов метода head()


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Вызов метода info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB

Вызов метода describe()


,count,mean,std,min,25%,50%,75%,max
book_id,1000.0,500.500,288.819436,1.0,250.75,500.5,750.25,1000.0
author_id,1000.0,320.417,181.620172,1.0,162.75,316.5,481.00,636.0
num_pages,1000.0,389.111,229.390140,14.0,249.00,352.0,453.00,2690.0
publisher_id,1000.0,171.270,99.082685,1.0,83.00,177.5,258.00,340.0



Количество пропусков: book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64
authors
Вызов метода head()


...


Количество пропусков: rating_id    0
book_id      0
username     0
rating       0
dtype: int64
publisher
Вызов метода head()


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



Вызов метода info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB

Вызов метода describe()


,count,mean,std,min,25%,50%,75%,max
review_id,2793.0,1397.000000,806.413976,1.0,699.0,1397.0,2095.0,2793.0
book_id,2793.0,504.693161,288.472931,1.0,259.0,505.0,753.0,1000.0



Количество пропусков: review_id    0
book_id      0
username     0
text         0
dtype: int64


Названия таблиц корректны и соответствуют описанию данных, пропусков нет, таблицы корректны. 

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [11]:
query = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE CAST(publication_date AS date) > '2000-01-01'

        '''
books_count = pd.io.sql.read_sql(query, con = engine) 
print('Количество книг, вышедшее после 1 января 2000 года: {}'.format(books_count.loc[0, 'count']))

Количество книг, вышедшее после 1 января 2000 года: 819


## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [12]:
query = '''
        SELECT b.title,
               COUNT(DISTINCT(rw.text)) AS count,
               AVG(rt.rating) AS avg_rating
        FROM books AS b
        LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
        LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
        GROUP BY b.book_id
        ORDER BY count DESC
        '''
books_review = pd.io.sql.read_sql(query, con = engine) 

In [13]:
books_review

,title,count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [14]:
query = '''
        SELECT p.publisher AS publisher_name,
               COUNT(b.book_id) AS count
        FROM publishers AS p
        JOIN books AS b ON p.publisher_id = b.publisher_id
        WHERE b.num_pages > 50
        GROUP BY publisher_name
        ORDER BY count DESC
        LIMIT 1

        '''

best_publisher = pd.io.sql.read_sql(query, con = engine) 

In [15]:
best_publisher

,publisher_name,count
0,Penguin Books,42


In [16]:
print('Издательство, выпустившее наибольшее количество книг: {}. Количество выпущенных книг: {}'.format(
      best_publisher.loc[0, 'publisher_name'], best_publisher.loc[0,'count']))

Издательство, выпустившее наибольшее количество книг: Penguin Books. Количество выпущенных книг: 42


## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [17]:
query = '''
        SELECT author AS author_name,
               AVG(r.rating) AS avg_rating
        FROM authors AS a
        JOIN books AS b ON a.author_id = b.author_id
        JOIN ratings AS r ON b.book_id = r.book_id
        GROUP BY author_name           
        HAVING COUNT(r.rating) > 50
        ORDER BY avg_rating DESC
        LIMIT 1
        '''
#PREV_best_author = pd.io.sql.read_sql(query, con = engine) 
#PREV_best_author

In [18]:
query = '''
        SELECT i.author AS author_name,
               AVG(i.rating_avg) as avg_rating
        FROM (SELECT author,
                     AVG(rating) AS rating_avg
              FROM authors AS a
              JOIN books AS b ON b.author_id = a.author_id
              JOIN ratings AS r ON r.book_id = b.book_id
              GROUP BY author,
                       b.book_id
              HAVING COUNT(rating) >= 50) AS i
        GROUP BY author_name
        ORDER BY avg_rating DESC
        LIMIT 1
        '''
best_author = pd.io.sql.read_sql(query, con = engine)

In [19]:
best_author

,author_name,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


In [20]:
print('Автор с самой высокой средней оценкой книг - {}. Средний рейтинг: {}'.format(
    best_author.loc[0, 'author_name'], round(best_author.loc[0,'avg_rating'], 2)))

Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré. Средний рейтинг: 4.28


## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [21]:
query = '''
        SELECT AVG(c.count) AS avg_review_count
        FROM (SELECT COUNT(r.username) AS count
             FROM reviews AS r
             GROUP BY r.username
             HAVING username IN (SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(rating_id) > 50
                               )) AS c

        '''

avg_reviews = pd.io.sql.read_sql(query, con = engine) 

In [22]:
avg_reviews

,avg_review_count
0,24.333333


In [23]:
print('Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: {}'.format(
        round(avg_reviews.loc[0, 'avg_review_count'])))

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24


**Вывод:**

- Количество книг, вышедшее после 1 января 2000 года: 819;
- Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books;
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24

Также мы выгрузили таблицу `books_review` с количеством обзоров на каждую книгу и их средние оценки. 